In [62]:
from sqlalchemy import create_engine, inspect
import pandas as pd

# Path to the database file
db_path = 'cademycode2.db'

# Create a database engine
engine = create_engine(f'sqlite:///{db_path}')

In [63]:
# inspector to get the list of tables
inspector = inspect(engine)
tables = inspector.get_table_names()

In [64]:
# Import each table into a DataFrame
dataframes = {}
if not tables:
    print("No tables found in the database.")
else:
    print("Tables in the database:")
    for table in tables:
        print(table)
        df = pd.read_sql_table(table, engine)
        dataframes[table] = df
        print(f"Data from {table}:")
        print(df.head())

# Now all the tables have been loaded into DataFrames stored in the `dataframes` dictionary

Tables in the database:
cademycode_courses
Data from cademycode_courses:
   career_path_id      career_path_name  hours_to_complete
0               1        data scientist                 20
1               2         data engineer                 20
2               3          data analyst                 12
3               4  software engineering                 25
4               5      backend engineer                 18
cademycode_student_jobs
Data from cademycode_student_jobs:
   job_id        job_category  avg_salary
0       1           analytics       86000
1       2            engineer      101000
2       3  software developer      110000
3       4            creative       66000
4       5  financial services      135000
cademycode_students
Data from cademycode_students:
   uuid             name         dob sex  \
0     1  Annabelle Avery  1943-07-03   F   
1     2      Micah Rubio  1991-02-07   M   
2     3       Hosea Dale  1989-12-07   M   
3     4     Mariann Kirk  1988-07-3

In [65]:
df_courses = dataframes.get('cademycode_courses', pd.DataFrame())
df_jobs = dataframes.get('cademycode_student_jobs', pd.DataFrame())
df_students = dataframes.get('cademycode_students', pd.DataFrame())
print(df_jobs)

    job_id        job_category  avg_salary
0        1           analytics       86000
1        2            engineer      101000
2        3  software developer      110000
3        4            creative       66000
4        5  financial services      135000
5        6           education       61000
6        7                  HR       80000
7        8             student       10000
8        9          healthcare      120000
9        0               other       80000
10       3  software developer      110000
11       4            creative       66000
12       5  financial services      135000


In [66]:
from sqlalchemy import MetaData, Table, Column, Integer, String, DateTime, Float, text
import json

# Clean and process df_students
df_students['job_id'] = pd.to_numeric(df_students['job_id'], errors='coerce')
df_students['num_course_taken'] = pd.to_numeric(df_students['num_course_taken'], errors='coerce')
df_students['current_career_path_id'] = pd.to_numeric(df_students['current_career_path_id'], errors='coerce')
df_students['time_spent_hrs'] = pd.to_numeric(df_students['time_spent_hrs'], errors='coerce')

print(df_students.head())

   uuid             name         dob sex  \
0     1  Annabelle Avery  1943-07-03   F   
1     2      Micah Rubio  1991-02-07   M   
2     3       Hosea Dale  1989-12-07   M   
3     4     Mariann Kirk  1988-07-31   F   
4     5  Lucio Alexander  1963-08-31   M   

                                        contact_info  job_id  \
0  {'mailing_address': '303 N Timber Key, Irondal...     7.0   
1  {'mailing_address': '767 Crescent Fair, Shoals...     7.0   
2  {'mailing_address': 'P.O. Box 41269, St. Bonav...     7.0   
3  {'mailing_address': '517 SE Wintergreen Isle, ...     6.0   
4  {'mailing_address': '18 Cinder Cliff, Doyles b...     7.0   

   num_course_taken  current_career_path_id  time_spent_hrs  
0               6.0                     1.0            4.99  
1               5.0                     8.0            4.40  
2               8.0                     8.0            6.74  
3               7.0                     9.0           12.31  
4              14.0                     

In [67]:
# Drop rows with NaN values
df_dropped = df_students.dropna(subset=['job_id', 'num_course_taken', 'current_career_path_id', 'time_spent_hrs'])
df_dropped.loc[:, 'job_id'] = df_dropped['job_id'].astype(int)
df_dropped.loc[:, 'num_course_taken'] = df_dropped['num_course_taken'].astype(int)
df_dropped.loc[:, 'current_career_path_id'] = df_dropped['current_career_path_id'].astype(int)
df_dropped.loc[:, 'time_spent_hrs'] = df_dropped['time_spent_hrs'].astype(float)
df_dropped.loc[:, 'contact_info'] = df_dropped['contact_info'].apply(json.dumps)

In [68]:
cleaned_df_jobs = df_jobs.drop_duplicates()
print(cleaned_df_jobs)

   job_id        job_category  avg_salary
0       1           analytics       86000
1       2            engineer      101000
2       3  software developer      110000
3       4            creative       66000
4       5  financial services      135000
5       6           education       61000
6       7                  HR       80000
7       8             student       10000
8       9          healthcare      120000
9       0               other       80000


In [70]:
# Define new table schema
metadata = MetaData()

cademycode_students2 = Table(
    'cademycode_students2',
    metadata,
    Column('uuid', Integer, primary_key=True),
    Column('name', String),
    Column('dob', String),
    Column('sex', String),
    Column('contact_info', String),
    Column('job_id', Integer),
    Column('num_course_taken', Integer),
    Column('current_career_path_id', Integer),
    Column('time_spent_hrs', Float)
)

# Create new table
metadata.create_all(engine)

In [71]:
# Insert data into new table
df_dropped.to_sql('cademycode_students2', engine, if_exists='replace', index=False)

# Drop old table and rename new table
with engine.connect() as conn:
    # Execute DROP TABLE statement
    drop_statement = text("DROP TABLE IF EXISTS cademycode_students")
    conn.execute(drop_statement)

    # Execute ALTER TABLE statement to rename
    rename_statement = text("ALTER TABLE cademycode_students2 RENAME TO cademycode_students")
    conn.execute(rename_statement)

# Verify changes
df_updated = pd.read_sql_table('cademycode_students', engine)

In [72]:
# Rename column
alter_query = text("ALTER TABLE cademycode_students RENAME COLUMN current_career_path_id TO career_path_id")

with engine.connect() as conn:
    conn.execute(alter_query)

In [73]:
# Load data after alteration
df_students = pd.read_sql("SELECT * FROM cademycode_students", engine)

In [74]:
merged_df = pd.merge(df_students, cleaned_df_jobs, on='job_id', how='inner')
merged_df = pd.merge(merged_df, df_courses, on='career_path_id', how='inner')

In [75]:
merged_df.to_csv('merged_data.csv', index=False)